In [ ]:
import sys
sys.path.append("../")
from utils import compile_df
from sbibm.utils.io import get_tensor_from_csv
import pandas as pd
import altair as alt
import deneb as den
from scipy.stats import binom

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import sbibm
import bz2
import _pickle as cPickle

from sbi.analysis import pairplot

In [ ]:
path = "2021-04-22/08-47-09/"
df = compile_df(basepath=path)

In [ ]:
# df = pd.read_csv(path + "/dataframe.csv")
df.to_csv(path + "/dataframe.csv")

In [ ]:
df.groupby(by=[df.algorithm, df.num_rounds, df.num_simulations]).cold_C2ST.mean()

In [ ]:
chart = den.lineplot(
    df, 
    x="num_simulations:O", 
    y="C2ST:Q",
#     column="algorithm:N",
    color=alt.Color("algorithm:N"),
    error_extent="stderr",
    width=500, 
    height=300)
chart

In [ ]:
chart = den.lineplot(
    df, 
    x="num_simulations:O", 
    y="cold_C2ST:Q",
#     column="algorithm:N",
    color=alt.Color("algorithm:N"),
    error_extent="stderr",
    width=500, 
    height=300)
chart

In [ ]:
chart = den.lineplot(
    df, 
    x="num_simulations:O", 
    y="cold_STD_ERR:Q",
#     column="algorithm:N",
    color=alt.Color("algorithm:N"),
    error_extent="stderr",
    width=500, 
    height=300)
chart

In [ ]:
chart = den.lineplot(
    df, 
    x="num_simulations:O", 
    y="cold_MEAN_ERR:Q",
#     column="algorithm:N",
    color=alt.Color("algorithm:N"),
    error_extent="stderr",
    width=500, 
    height=300)
chart

In [ ]:
chart = den.lineplot(
    df, 
    x="num_rounds:O", 
    y="MAP/MLE_ERR:Q",
#     column="algorithm:N",
    color=alt.Color("algorithm:N"),
    error_extent="stderr",
    width=500, 
    height=300, )
chart

In [ ]:
chart = den.lineplot(
    df, 
    x="num_rounds:O", 
    y="MEAN_ERR:Q",
#     column="algorithm:N",
    color=alt.Color("algorithm:N"),
    error_extent="stderr",
    width=500, 
    height=300)
chart

In [ ]:
task = sbibm.get_task("gaussian_linear_uniform")

In [ ]:
import torch
from sbibm.metrics import c2st

In [ ]:
nr = [1, 1, 2, 2, 4, 4, 6, 6, 8, 8, 10, 10]
for i in range(12):
    
    true_cold_samples = []
    sampling_dist = torch.distributions.MultivariateNormal(
        loc=task.get_observation(1).squeeze(),
        precision_matrix=task.simulator_params["precision_matrix"] * nr[i],
    )

    # Reject samples outside of prior bounds
    counter = 0
    num_accepted = 0
    while num_accepted < 1000:
        counter += 1
        sample = sampling_dist.sample((10000,))
        within_support = task.prior_dist.support.check(sample)
        if within_support.any():
            true_cold_samples.append(sample[within_support])
            num_accepted += int(within_support.sum())
    s1 = torch.cat(true_cold_samples)[:1000]
    s2 = get_tensor_from_csv(f"2021-04-17/09-05-26/{i}/posterior_samples.csv.bz2")[:1000,]
#     s2 = task.get_reference_posterior_samples(i+1)[:1000]
#     print(c2st(s1, s2))
    pairplot([
        s1,
        s2, 
    ], upper="scatter", limits=[[-3, 3]]*10,
             points=task.get_observation(1), points_colors=["C2"], figsize=(18, 12), 
        hist_diag=dict(bins=np.linspace(-2, 2, 30)));
    plt.suptitle(f"num_rounds {nr[i]}", fontsize=15)